<a href="https://colab.research.google.com/github/AlexBugalter/Lesson/blob/main/lstm_prediction_eurusd_for_d1_input_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Этот скрипт использует библиотеку TensorFlow для создания и обучения модели LSTM (Long Short-Term Memory) для прогнозирования цен на валютную пару EURUSD на дневных временных интервалах (D1). Модель принимает на вход последовательность из 10 предыдущих баров (открытие, максимум, минимум, закрытие) и пытается предсказать цену закрытия следующего бара.

После обучения модель сохраняется в формате ONNX для последующего использования.

In [ ]:
!pip install tf2onnx # Эта команда установит tf2onnx из репозитория PyPI.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 40.9 MB/s eta 0:00:00


In [ ]:
# Импортируем необходимые библиотеки
from datetime import datetime
import yfinance as yf
import tensorflow as tf
import numpy as np
import pandas as pd
import tf2onnx
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sys import argv

In [ ]:
# Определяем путь для сохранения созданной ONNX модели
data_path = argv[0]
last_index = data_path.rfind("\\") + 1
data_path = data_path[0:last_index]
print("Путь к данным для сохранения onnx модели:", data_path)

Путь к данным для сохранения onnx модели: 


In [ ]:
# Входные параметры
inp_model_name = "model.eurusd.D1.10.onnx"  # Имя файла для сохранения ONNX модели
inp_history_size = 10  # Количество предыдущих баров для прогнозирования
inp_start_date = datetime(2003, 1, 1, 0)  # Начальная дата для получения данных
inp_end_date = datetime(2023, 1, 1, 0)  # Конечная дата для получения данных

In [ ]:
# Получаем данные 
eurusd = yf.download("EURUSD=X", start=inp_start_date, end=inp_end_date, interval="1d")
df = eurusd.reset_index()[["Date", "Open", "High", "Low", "Close"]]
df.columns = ["time", "open", "high", "low", "close"]
# df = pd.DataFrame(eurusd_rates)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
print("Длина DataFrame:", len(df))

Длина DataFrame: 4951


In [ ]:
# Функция для сбора набора данных
def collect_dataset(df: pd.DataFrame, history_size: int):
    """
    Сбор набора данных для последующей задачи регрессии:
    - ввод: history_size последовательных баров D1;
    - вывод: цена закрытия для следующего бара.

    :param df: D1 бары для определенного периода времени
    :param history_size: сколько баров следует учитывать для прогнозирования
    :return: признаки и метки
    """
    n = len(df)
    xs = []
    ys = []
    for i in tqdm(range(n - history_size + 1)):
        if i + history_size < n:
            x = df.loc[i:i+history_size-1, ['open', 'high', 'low', 'close']].values
            y = df.loc[i+history_size, 'close']
            xs.append(x)
            ys.append(y)
    X = np.array(xs)
    y = np.array(ys)
    return X, y

In [ ]:
# Получаем признаки и метки из набора данных
X, y = collect_dataset(df, history_size=inp_history_size)

100%|██████████| 4942/4942 [00:04<00:00, 1220.38it/s]


In [ ]:
# Нормализуем признаки и метки
m = X.mean(axis=1, keepdims=True)
s = X.std(axis=1, keepdims=True)
X_norm = (X - m) / s
y_norm = (y - m[:, 0, 3]) / s[:, 0, 3]

In [ ]:
# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=0)

In [ ]:
# Определяем архитектуру модели
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(inp_history_size, 4)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Компилируем модель
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# Обучаем модель на 50 эпохах с уменьшением скорости обучения
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.000001)
history = model.fit(X_train, y_train, epochs=50, verbose=2, validation_split=0.15, callbacks=[lr_reduction])

Epoch 1/50
105/105 - 5s - loss: 1.6473 - mae: 0.9215 - val_loss: 2.3919 - val_mae: 1.2421 - lr: 0.0010 - 5s/epoch - 50ms/step
Epoch 2/50
105/105 - 1s - loss: 1.2598 - mae: 0.7903 - val_loss: 2.2294 - val_mae: 1.1829 - lr: 0.0010 - 894ms/epoch - 9ms/step
Epoch 3/50
105/105 - 1s - loss: 1.1897 - mae: 0.7590 - val_loss: 1.8157 - val_mae: 1.0308 - lr: 0.0010 - 1s/epoch - 10ms/step
Epoch 4/50
105/105 - 1s - loss: 1.1058 - mae: 0.7267 - val_loss: 1.4077 - val_mae: 0.8791 - lr: 0.0010 - 971ms/epoch - 9ms/step
Epoch 5/50
105/105 - 1s - loss: 1.1146 - mae: 0.7343 - val_loss: 1.0899 - val_mae: 0.7230 - lr: 0.0010 - 1s/epoch - 12ms/step
Epoch 6/50
105/105 - 1s - loss: 1.0918 - mae: 0.7236 - val_loss: 1.0022 - val_mae: 0.6902 - lr: 0.0010 - 1s/epoch - 11ms/step
Epoch 7/50
105/105 - 1s - loss: 1.1180 - mae: 0.7299 - val_loss: 1.0276 - val_mae: 0.6984 - lr: 0.0010 - 970ms/epoch - 9ms/step
Epoch 8/50
105/105 - 1s - loss: 1.0625 - mae: 0.7080 - val_loss: 0.9507 - val_mae: 0.6629 - lr: 0.0010 - 970ms/e

In [ ]:
# Оцениваем модель на тестовом наборе
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"test_loss={test_loss:.3f}")
print(f"test_mae={test_mae:.3f}")

31/31 [==============================] - 0s 6ms/step - loss: 0.8971 - mae: 0.6205
test_loss=0.897
test_mae=0.621


In [ ]:
# Сохраняем модель в формате ONNX
output_path = data_path + inp_model_name
onnx_model = tf2onnx.convert.from_keras(model, output_path=output_path)
print(f"Сохраненная модель в {output_path}")

Сохраненная модель в model.eurusd.D1.10.onnx
